In [1]:
import entsoe_client as ec
from entsoe_client import ParameterTypes
import pandas as pd

In [3]:
# Instantiate client implementing basic GET Request interface to ENTSOE API.
# Requires an API Key for access.
api_key = "6e68642c-8403-4caa-af31-bda40b8c67f6"
client = ec.Client(api_key)

# Query settings.
periodStart = 202206010015
periodEnd = 202206010230
in_Domain = ec.ParameterTypes.Area('DE_LU')
outBiddingZone_Domain = in_Domain

# Set up parser to parse XML scheme into pandas dataframe.
parser = ec.Parser()

# Query actual aggregated generation per type.
query_gen = ec.Queries.Generation.AggregatedGenerationPerType(
    in_Domain=in_Domain,
    periodStart=periodStart,
    periodEnd=periodEnd)

# Query actual total load.
query_load = ec.Queries.Load.ActualTotalLoad(
    outBiddingZone_Domain,
    periodStart=periodStart,
    periodEnd=periodEnd)

In [5]:
# Extract data.
response_gen = client(query_gen)
response_load = client(query_load)
df_gen = parser.parse(response_gen)
df_load = parser.parse(response_load)
#df.to_csv("./test.csv")

In [9]:
print(df_load.columns)

Index(['position', 'quantity', 'Period.timeInterval.start',
       'Period.timeInterval.end', 'Period.resolution', 'TimeSeries.mRID',
       'TimeSeries.businessType', 'TimeSeries.objectAggregation',
       'TimeSeries.outBiddingZone_Domain.mRID',
       'TimeSeries.quantity_Measure_Unit.name', 'TimeSeries.curveType',
       'GL_MarketDocument.mRID', 'GL_MarketDocument.revisionNumber',
       'GL_MarketDocument.type', 'GL_MarketDocument.process.processType',
       'GL_MarketDocument.sender_MarketParticipant.mRID',
       'GL_MarketDocument.sender_MarketParticipant.marketRole.type',
       'GL_MarketDocument.receiver_MarketParticipant.mRID',
       'GL_MarketDocument.receiver_MarketParticipant.marketRole.type',
       'GL_MarketDocument.createdDateTime',
       'GL_MarketDocument.time_Period.timeInterval.start',
       'GL_MarketDocument.time_Period.timeInterval.end'],
      dtype='object')


In [11]:
consumption_mask = df_gen["TimeSeries.outBiddingZone_Domain.mRID"].notna()
# mRID: Document ID of the submitted Status Request Document. The mRID can be used for matching requests with responses.
print(consumption_mask)
production = df_gen[~consumption_mask][["quantity", "TimeSeries.MktPSRType.psrType"]]
#print(production.iloc[:,:10])
#print(df.iloc[:,:10])
#print(df.columns)
#print(consumption_mask)
#print(production)

2022-06-01 00:15:00+00:00    False
2022-06-01 00:30:00+00:00    False
2022-06-01 00:45:00+00:00    False
2022-06-01 01:00:00+00:00    False
2022-06-01 01:15:00+00:00    False
                             ...  
2022-06-01 01:15:00+00:00     True
2022-06-01 01:30:00+00:00     True
2022-06-01 01:45:00+00:00     True
2022-06-01 02:00:00+00:00     True
2022-06-01 02:15:00+00:00     True
Name: TimeSeries.outBiddingZone_Domain.mRID, Length: 171, dtype: bool


In [7]:
production["GenerationType"] = production["TimeSeries.MktPSRType.psrType"].apply(
    lambda x: ParameterTypes.PsrType[x].value
)  # Use ParameterTypes to transform ENTSO-E Code into readable string.
production_by_type = pd.pivot_table(
    production, index=production.index, columns="GenerationType", values="quantity"
)

In [8]:
print(production.shape)

(144, 3)


In [7]:
print(production_by_type)
print(production_by_type.columns, production_by_type.shape)

GenerationType             Biomass  Fossil Brown coal / Lignite  Fossil Gas  \
2022-06-01 00:15:00+00:00   4411.0                      14696.0      7017.0   
2022-06-01 00:30:00+00:00   4404.0                      14723.0      7058.0   
2022-06-01 00:45:00+00:00   4388.0                      14634.0      7073.0   
2022-06-01 01:00:00+00:00   4370.0                      14614.0      7177.0   
2022-06-01 01:15:00+00:00   4379.0                      14577.0      7143.0   
2022-06-01 01:30:00+00:00   4391.0                      14598.0      7148.0   
2022-06-01 01:45:00+00:00   4397.0                      14603.0      7149.0   
2022-06-01 02:00:00+00:00   4409.0                      14600.0      7207.0   
2022-06-01 02:15:00+00:00   4414.0                      14600.0      7190.0   

GenerationType             Fossil Hard coal  Fossil Oil  Geothermal  \
2022-06-01 00:15:00+00:00            4328.0         8.0        25.0   
2022-06-01 00:30:00+00:00            4283.0         8.0        25.0